In [62]:
from classiq import (
    Model,
    RegisterUserInput,
    FunctionLibrary,
    QASM_INTRO,
    qfunc,
    QReg,
)
from classiq.builtin_functions import ArithmeticOracle, GroverOperator, CustomOracle
from classiq import synthesize, show

In [63]:
@qfunc
def my_state_preparation(args: QReg[18]) -> QReg[18]:
    return QASM_INTRO + "qreg q[18];"

@qfunc
def my_oracle(args: QReg[18]) -> QReg[18]:
    return QASM_INTRO + "qreg q[18];"

my_function_library = FunctionLibrary(my_state_preparation, my_oracle)

In [64]:
old_oracle_params = ArithmeticOracle(
    expression="(a + b + (c & 6)) % 4 | 4 & c  == 4",
    definitions=dict(
        a=RegisterUserInput(size=2),
        b=RegisterUserInput(size=2),
        c=RegisterUserInput(size=3),
    ),
    uncomputation_method="optimized",
    qubit_count=16,
)

In [66]:
new_oracle_params = CustomOracle(
    custom_oracle= "my_oracle",
    custom_oracle_params= my_function_library.get_function("my_oracle"),
)

In [67]:
grover_params = GroverOperator(
    oracle_params= new_oracle_params,
    state_preparation = "my_state_preparation",
    state_preparation_params = my_function_library.get_function("my_state_preparation"),
)

In [68]:
model = Model()
model.include_library(my_function_library)
model.GroverOperator(grover_params)
model = model.get_model()

quantum_program = synthesize(model)
show(quantum_program)

Opening: https://platform.classiq.io/circuit/7eed4bd8-cc72-4c6b-bd8d-a7d56838ee30?version=0.33.0
